In [2]:
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.models import Model

In [3]:
print(tf.__version__)
print(keras.__version__)

2.5.3
2.5.0


In [4]:
(x_train, y_train) , (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [5]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [9]:
with strategy.scope():
    input_x = keras.layers.Input(shape=(28,28,1))

    x = Conv2D(64, kernel_size=3, activation='relu')(input_x)
    x = BatchNormalization()(x)
    x = Conv2D(32, kernel_size=3, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(16, kernel_size=3, activation='relu')(x)

    x = Flatten()(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_y = Dense(10, activation='softmax')(x)

    model = Model(inputs = input_x, outputs = output_y)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=256)

Epoch 1/5


2023-07-16 15:52:16.249541: W tensorflow/core/framework/dataset.cc:807] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


1875/1875 [==============================] - 44s 23ms/step - loss: 0.2968 - accuracy: 0.9145
Epoch 2/5
1131/1875 [=================>............] - ETA: 17s - loss: 0.1499 - accuracy: 0.9566

In [38]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 4s - loss: 0.0762 - accuracy: 0.9783 - 4s/epoch - 13ms/step


[0.0761503353714943, 0.9782999753952026]